In [93]:
import argparse
import torch
import numpy as np
import json, io
from nltk import word_tokenize
import nltk
import os
import xml.etree.ElementTree as ET
from keras.utils import to_categorical

In [94]:
import fasttext
from fasttext import load_model

In [95]:
class Model(torch.nn.Module):
    def __init__(self, gen_emb, domain_emb, num_classes=3, dropout=0.5, crf=False, tag=False):
        super(Model, self).__init__()
        self.tag_dim = 45 if tag else 0

        self.gen_embedding = torch.nn.Embedding(gen_emb.shape[0], gen_emb.shape[1])
        self.gen_embedding.weight=torch.nn.Parameter(torch.from_numpy(gen_emb), requires_grad=False)
        self.domain_embedding = torch.nn.Embedding(domain_emb.shape[0], domain_emb.shape[1])
        self.domain_embedding.weight=torch.nn.Parameter(torch.from_numpy(domain_emb), requires_grad=False)
    
        self.conv1=torch.nn.Conv1d(gen_emb.shape[1]+domain_emb.shape[1], 128, 5, padding=2 )
        self.conv2=torch.nn.Conv1d(gen_emb.shape[1]+domain_emb.shape[1], 128, 3, padding=1 )
        self.dropout=torch.nn.Dropout(dropout)

        self.conv3=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv4=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv5=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.linear_ae1=torch.nn.Linear(256+self.tag_dim+domain_emb.shape[1], 50)
        self.linear_ae2=torch.nn.Linear(50, num_classes)
        self.crf_flag=crf
        if self.crf_flag:
            from allennlp.modules import ConditionalRandomField
            self.crf=ConditionalRandomField(num_classes)            
          
    def forward(self, x, x_len, x_mask, x_tag, y=None, testing=False):
        x_emb=torch.cat((self.gen_embedding(x), self.domain_embedding(x) ), dim=2)  # shape = [batch_size (128), sentence length (83), embedding output size (300+100)]       
        x_emb=self.dropout(x_emb).transpose(1, 2)  # shape = [batch_size (128), embedding output size (300+100+tag_num) , sentence length (83)]
        x_conv=torch.nn.functional.relu(torch.cat((self.conv1(x_emb), self.conv2(x_emb)), dim=1) )  # shape = [batch_size, 128+128, 83]
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv3(x_conv) )
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv4(x_conv) )
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv5(x_conv) )
        x_conv=x_conv.transpose(1, 2) # shape = [batch_size, 83, 256]
        x_logit=torch.nn.functional.relu(self.linear_ae1(torch.cat((x_conv, x_tag, self.domain_embedding(x)), dim=2) ) ) # shape = [batch_size, 83, 20]
        x_logit=self.linear_ae2(x_logit)
        if testing:
            if self.crf_flag:
                score=self.crf.viterbi_tags(x_logit, x_mask)
            else:
                x_logit=x_logit.transpose(2, 0)
                score=torch.nn.functional.log_softmax(x_logit).transpose(2, 0)
        else:
            if self.crf_flag:
                score=-self.crf(x_logit, y, x_mask)
            else:
                x_logit=torch.nn.utils.rnn.pack_padded_sequence(x_logit, x_len, batch_first=True)
                score=torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(x_logit.data), y.data)
        return score
    

In [96]:
class bcolors:
    ONGREEN = '\x1b[6;30;42m'
    ONYELLOW = '\x1b[6;30;46m'
    ONRED = '\x1b[6;30;41m'
    ONPURPLE = '\x1b[6;30;45m'
    END = '\x1b[0m'

In [97]:
def build_emb_dictionary(fn):
    words=[]
    vectors=[]
    with open(fn) as f:
        for l in f:
            t=l.rstrip().split(' ')
            words.append(t[0])
            vectors.append(list(map(float, t[1:])))
        wordvecs = np.array(vectors, dtype=np.double)
        word2id = {word:i for i, word in enumerate(words)}
    return wordvecs,word2id

In [98]:
def prep_emb(fn, gen_emb, domain_emb, prep_dir, gen_dim=300, domain_dim=100):
    text = []
    with open(fn) as f:
        for line in f:
            token = word_tokenize(line)
            text=text+token
    vocab = sorted(set(text))
    word_idx = {}
    if os.path.exists(prep_dir+'word_idx.json'):
        with io.open(prep_dir+'word_idx.json') as f:
            prev_word = json.load(f)
    else:
        prev_word = {}
    wx = 0
    new_word = []
    for word in vocab:
        if word not in prev_word:
            wx = wx+1
            new_word.append(word)
            word_idx[word] = wx+len(prev_word)
    prev_word.update(word_idx)          
    if new_word == []:
        return
    # create embedding
    embedding_gen=np.zeros((len(prev_word)+2, gen_dim) )
    embedding_domain=np.zeros((len(prev_word)+2, domain_dim) )    
    if os.path.exists(prep_dir+'glove.840B.300d.txt.npy'):
        gen_emb_prev=np.load(prep_dir+"glove.840B.300d.txt.npy")
        embedding_gen[:gen_emb_prev.shape[0],:] = gen_emb_prev
    if os.path.exists(prep_dir+'glove.840B.300d.txt.npy'):
        domain_emb_prev=np.load(prep_dir+'restaurant_emb.vec.npy')
        embedding_domain[:domain_emb_prev.shape[0],:] = domain_emb_prev
    with open(gen_emb) as f:
        # read the embedding .vec file
        for l in f:
            rec=l.rstrip().split(' ')
            if len(rec)==2: #skip the first line.
                continue 
            # if the word in word_idx, fill the embedding
            if rec[0] in new_word:
                embedding_gen[prev_word[rec[0]]] = np.array([float(r) for r in rec[1:] ])
    with open(domain_emb) as f:
        # read the embedding .vec file
        for l in f:
            # for each line, get the word and its vector
            rec=l.rstrip().split(' ')
            if len(rec)==2: #skip the first line.
                continue 
            # if the word in word_idx, fill the embedding
            if rec[0] in new_word:
                embedding_domain[prev_word[rec[0]]] = np.array([float(r) for r in rec[1:] ])
    ftmodel = load_model(domain_emb+".bin")
    for w in new_word:
        if embedding_domain[word_idx[w] ].sum()==0.:
            embedding_domain[word_idx[w] ] = ftmodel.get_word_vector(w)
    with io.open(prep_dir+'word_idx.json', 'w') as outfile:
        outfile.write(json.dumps(prev_word)) 
    np.save(prep_dir+'glove.840B.300d.txt.npy', embedding_gen.astype('float32') )
    np.save(prep_dir+'restaurant_emb.vec.npy', embedding_domain.astype('float32') )  

In [99]:
def prep_text(fn, POSdir, prep_dir):
        # map part-of-speech tag to int
    pos_tag_list = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS','NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP','SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB',',','.',':','$','#',"``","''",'(',')']
    tag_to_num = {tag:i+1 for i, tag in enumerate(sorted(pos_tag_list))}

    with io.open(prep_dir+'word_idx.json') as f:
        word_idx=json.load(f)
    sentence_size = [-1, 130]
    with open(fn) as f:
        count = 0
        for l in f:
            token = word_tokenize(l)
            if len(token) > 0:
                count = count + 1
                if len(token) > sentence_size[1]:
                    sentence_size[1]=len(token)
        sentence_size[0] = count
        X = np.zeros((sentence_size[0], sentence_size[1]), np.int16)
        X_tag = np.zeros((sentence_size[0], sentence_size[1]), np.int16)
     
    with open(fn) as f:   
        count = -1
        raw_X=[]
        for l in f:
            token = word_tokenize(l)
            #jar = POSdir+'stanford-postagger.jar'
            #model = POSdir+'models/english-left3words-distsim.tagger'
            #pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')
            pos_tag_stf = [tag_to_num[tag] for (_,tag) in nltk.pos_tag(token)]
            if len(token) > 0:
                count = count + 1
                raw_X.append(token)   
                # write word index and tag in train_X and train_X_tag
                for wx, word in enumerate(token):
                    X[count, wx] = word_idx[word]
                    X_tag[count, wx] = pos_tag_stf[wx]
    return raw_X, X, X_tag

In [156]:
theDict=[('\0','\0')]
def output_text(fn, pred_y, out_fn=None):
    count = -1
    with open(fn) as f:
        for l in f:
            token = word_tokenize(l)
            if len(token) > 0:
                count = count + 1
                #print(token)
                AWL=[] #the Aspect Word List
                sentL=[] #Single Sentence List
                for wx, word in enumerate(token):

                    if pred_y[count, wx] == 1:
                        print(bcolors.ONGREEN + word + bcolors.END, end=" ")
                        #print(bcolors.ONGREEN + 'this' + bcolors.END, end=" ")
                        AWL.append(word)
                        sentL.append(word)
                        #print(word,end=' ')
                    elif pred_y[count, wx] == 2:
                        print(bcolors.ONGREEN + word + bcolors.END, end=" ")
                        AWL.append(word)
                        sentL.append(word)
                        #print(word,end=' ')
                    else:
                        print(word, end=" ")
                        sentL.append(word)
                    #print('\n')
                compsent=(" ".join(sentL)) #complete sentence
                print(AWL)
                print(compsent)
                theDict.append((compsent,AWL))
                print("\n")
            else:
                print('\n') 

In [150]:
def test(model, test_X, test_X_tag, raw_X, domain, batch_size=128, crf=False, tag=False):
    pred_y=np.zeros((test_X.shape[0], test_X.shape[1]), np.int16)
    model.eval()
    for offset in range(0, test_X.shape[0], batch_size):
        batch_test_X_len=np.sum(test_X[offset:offset+batch_size]!=0, axis=1)
        batch_idx=batch_test_X_len.argsort()[::-1]
        batch_test_X_len=batch_test_X_len[batch_idx]
        batch_test_X_mask=(test_X[offset:offset+batch_size]!=0)[batch_idx].astype(np.uint8)
        batch_test_X=test_X[offset:offset+batch_size][batch_idx]
        batch_test_X_mask=torch.autograd.Variable(torch.from_numpy(batch_test_X_mask).long() )
        batch_test_X = torch.autograd.Variable(torch.from_numpy(batch_test_X).long() )
        if tag:
            batch_test_X_tag = test_X_tag[offset:offset+batch_size][batch_idx]
            batch_test_X_tag_onehot = to_categorical(batch_test_X_tag, num_classes=45+1)[:,:,1:]
            batch_test_X_tag_onehot = torch.autograd.Variable(torch.from_numpy(batch_test_X_tag_onehot).type(torch.FloatTensor) )
        else:
            batch_test_X_tag_onehot = None
        batch_pred_y=model(batch_test_X, batch_test_X_len, batch_test_X_mask, batch_test_X_tag_onehot, testing=True)
        r_idx=batch_idx.argsort()
        if crf:
            batch_pred_y=[batch_pred_y[idx] for idx in r_idx]
            for ix in range(len(batch_pred_y) ):
                for jx in range(len(batch_pred_y[ix]) ):
                    pred_y[offset+ix,jx]=batch_pred_y[ix][jx]
        else:
            batch_pred_y=batch_pred_y.data.cpu().numpy().argmax(axis=2)[r_idx]
            pred_y[offset:offset+batch_size,:batch_pred_y.shape[1]]=batch_pred_y
    model.train()
    assert len(pred_y)==len(test_X)    
    return pred_y

In [151]:
def demo_by_input(demo_dir, demo_fn, model_fn, POSdir, domain, gen_emb, domain_emb, runs, gen_dim, domain_dim, prep_dir, crf=False, tag=True):           
    fn = demo_dir + demo_fn
    prep_emb(fn, gen_emb, domain_emb, prep_dir, gen_dim, domain_dim)
    raw_X, X, X_tag = prep_text(fn, POSdir, prep_dir)
    
    for run in range(runs):
        model_fn_run=model_fn+str(run)
        model=torch.load(model_fn_run, map_location=lambda storage, loc: storage)
        embedding_gen=np.load(prep_dir+"glove.840B.300d.txt.npy")
        embedding_domain=np.load(prep_dir+"restaurant_emb.vec.npy")
        model.gen_embedding = torch.nn.Embedding(embedding_gen.shape[0], embedding_gen.shape[1])
        model.gen_embedding.weight=torch.nn.Parameter(torch.from_numpy(embedding_gen).type(torch.FloatTensor), requires_grad=False)
        model.domain_embedding = torch.nn.Embedding(embedding_domain.shape[0], embedding_domain.shape[1])
        model.domain_embedding.weight=torch.nn.Parameter(torch.from_numpy(embedding_domain).type(torch.FloatTensor), requires_grad=False)        
        if not tag:
            model.tag_dim = 0
        
        pred_y = test(model, X, X_tag, raw_X, domain, crf=crf, tag=tag)
        output_text(fn, pred_y)

In [154]:
demo_by_input("data/official data/","TEST_data.txt","model/restaurant","stanford-postagger-full/","restaurant","data/embedding/glove.840B.300d.txt","data/embedding/restaurant_emb.vec",3,300,100,"demo/prep1/",False,True)

/Users/malikagupta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Yum ! []
Yum !


Serves really good sushi . ['Serves', 'sushi']
Serves really good sushi .


Not the biggest portions but adequate . ['portions']
Not the biggest portions but adequate .


Green Tea creme brulee is a must ! ['Green', 'Tea', 'creme']
Green Tea creme brulee is a must !


Do n't leave the restaurant without it . []
Do n't leave the restaurant without it .


No Comparison []
No Comparison


– I ca n't say enough about this place . ['place']
– I ca n't say enough about this place .


It has great sushi and even better service . ['sushi', 'service']
It has great sushi and even better service .


The entire staff was extremely accomodating and tended to my every need . ['entire']
The entire staff was extremely accomodating and tended to my every need .


I 've been to this restaurant over a dozen times with no complaints to date . ['restaurant']
I 've been to this restaurant over a dozen times with no complaints to date .


Snotty Attitude ['Attitude']
Snotty Attitude


– We w

Took forever to get our order taken , water refills were too much to ask for and the only time she was fast was when we asked for our bill when we could get her attention . []
Took forever to get our order taken , water refills were too much to ask for and the only time she was fast was when we asked for our bill when we could get her attention .


I do n't know if I 'll be back ... . []
I do n't know if I 'll be back ... .


Mama Mia – I live in the neighborhood and feel lucky to live by such a great pizza place . ['pizza']
Mama Mia – I live in the neighborhood and feel lucky to live by such a great pizza place .


the only problem is you really have to warm up the pizza before it 's edible , even when you order ahead . ['pizza']
the only problem is you really have to warm up the pizza before it 's edible , even when you order ahead .


BUt once done , it 's not too much dough , not too much cheese , not too much sauce . []
BUt once done , it 's not too much dough , not too much chees

It 's located in a strip mall near the Beverly Center , not the greatest location , but the food keeps me coming back for more . ['location', 'food']
It 's located in a strip mall near the Beverly Center , not the greatest location , but the food keeps me coming back for more .


hidden little jem []
hidden little jem


– tucked away over by the Beverly Center . []
– tucked away over by the Beverly Center .


Never too crowded and always great service . ['service']
Never too crowded and always great service .


I think I have probably tried each item on their menu at least once it is all excellent . []
I think I have probably tried each item on their menu at least once it is all excellent .


I can highly recommend their various saag and paneer and korma . ['various', 'saag', 'and', 'paneer', 'and', 'korma']
I can highly recommend their various saag and paneer and korma .


You routinely see Indian families and friends dining there , which is always a good sign . []
You routinely see I

What may be interesting to most is the worst sevice/attitude comes from the owners of this establishment . ['sevice/attitude', 'establishment']
What may be interesting to most is the worst sevice/attitude comes from the owners of this establishment .


This establishment really made a marked decline after ( and this is recurring story ) the airing of FOOD TELEVISIONS `` DINERS , DRIVE-INS , AND DIVES '' hosted by Guy Fieri , in which Schooner or Later was subject of . ['establishment', 'FOOD']
This establishment really made a marked decline after ( and this is recurring story ) the airing of FOOD TELEVISIONS `` DINERS , DRIVE-INS , AND DIVES '' hosted by Guy Fieri , in which Schooner or Later was subject of .


Perhaps now , Scooner or Later falls into the `` Dive '' category . []
Perhaps now , Scooner or Later falls into the `` Dive '' category .


This is sad for what once was one of the best places you could ever eat . []
This is sad for what once was one of the best places you coul

...

melt in your mouth . []
melt in your mouth .


we are for sure coming back to this restaurant . []
we are for sure coming back to this restaurant .


every month if possible . []
every month if possible .


it is sad there is not many people who frequent eating at places like these that look pricey because they are at the hotel ... but they are definitely one you dont want to miss , esp if your in downtown san jose . []
it is sad there is not many people who frequent eating at places like these that look pricey because they are at the hotel ... but they are definitely one you dont want to miss , esp if your in downtown san jose .


Chintzy portions []
Chintzy portions


– The sushi here is perfectly good , but for $ 5 a piece , either the slices of fish should be larger , or there should be no pretense that this is a moderately priced restaurant ( even for NYC ) . []
– The sushi here is perfectly good , but for $ 5 a piece , either the slices of fish should be larger , or there should 

After one member of our party had been bumped repeatedly by a waitress , a polite request that he not be bumped sent the waitress into an abusive rant .


A brief conversation with the manager at the end of the meal was the greatest disappointment -- to say we had been `` blown off '' would be an understatement . []
A brief conversation with the manager at the end of the meal was the greatest disappointment -- to say we had been `` blown off '' would be an understatement .


The manager continually interrupted with `` Is there anything else I can do for you ? `` , a strange comment because she had hardly listened , let alone responded to our expression of disappointment at our experience . []
The manager continually interrupted with `` Is there anything else I can do for you ? `` , a strange comment because she had hardly listened , let alone responded to our expression of disappointment at our experience .


She promised to speak to the waitress who had flown off in a rage , but we co

Overall , I would go back and eat at the restaurant again . []
Overall , I would go back and eat at the restaurant again .


Just remember to dress a little urban-chic so you don ? t look too far out of place . []
Just remember to dress a little urban-chic so you don ? t look too far out of place .


good sake , good food – i honestly do n't know much about japanese food at all . []
good sake , good food – i honestly do n't know much about japanese food at all .


In the past sake usuallly just tasted kind of like salty water to me . []
In the past sake usuallly just tasted kind of like salty water to me .


Server made several sake suggestions which were very good . []
Server made several sake suggestions which were very good .


had many dishes but the BEST was the lobster 3 ways . []
had many dishes but the BEST was the lobster 3 ways .


VERY GOOD ! []
VERY GOOD !


Overpriced []
Overpriced


– We dined at the Colorado location today . []
– We dined at the Colorado location today .

The sushi is as fresh as it comes ? you 'd think ocean was in their backyard , no joke ! []
The sushi is as fresh as it comes ? you 'd think ocean was in their backyard , no joke !


If you 're interested in good tasting ( without the fish taste or smell ) , large portions and creative sushi dishes this is your place ... []
If you 're interested in good tasting ( without the fish taste or smell ) , large portions and creative sushi dishes this is your place ...


big thick pepperoni []
big thick pepperoni


– The pepperoni 's cut real thick -- Yum . []
– The pepperoni 's cut real thick -- Yum .


The pizza itself is not exactly the best I 've had EVER , but still pretty good . []
The pizza itself is not exactly the best I 've had EVER , but still pretty good .


I probably would n't go back though 'cuz I do n't know if it 's worth it . []
I probably would n't go back though 'cuz I do n't know if it 's worth it .


Maybe I 'll go back once more many years from now when I 've forgotten I

For my entr & eacute ; e , I completely enjoyed the seared Alaskan sea scallops complemented by chard , artichoke hearts , fennel , and pecorino toscano . []
For my entr & eacute ; e , I completely enjoyed the seared Alaskan sea scallops complemented by chard , artichoke hearts , fennel , and pecorino toscano .


The scallops are apparently cooked in a black olive butter which really makes them unique ( not to mention tasty ) . []
The scallops are apparently cooked in a black olive butter which really makes them unique ( not to mention tasty ) .


My friend enjoyed the grilled Alaskan King Salmon with delectable creamed Washington russet potatoes and crisp green beans . []
My friend enjoyed the grilled Alaskan King Salmon with delectable creamed Washington russet potatoes and crisp green beans .


I had a taste of all three items on her plate , and they were superb . []
I had a taste of all three items on her plate , and they were superb .


Our server continued to be attentive through

In [155]:
print(theDict)

[('\x00', '\x00'), ('Green Tea creme brulee is a must !', ['Green', 'Tea', 'creme']), ('It has great sushi and even better service .', ['sushi', 'service']), ('Not the biggest portions but adequate .', ['portions']), ('The owner is belligerent to guests that have a complaint .', ['owner']), ('The staff is pretty friendly .', ['staff']), ('The prices are outrageous , especially since the food was actually less satisfying than most neighborhood Chinese establishments .', ['food']), ('Nice ambience , but highly overrated place .', ['ambience', 'place']), ('Furthermore , while the fish is unquestionably fresh , rolls tend to be inexplicably bland .', ['fish', 'rolls']), ('On a recent trip , our waiter was extremely dismissive , while no less than three staff members waited hand-and-foot on a pair of Japanese girls seated nearby .', ['waiter']), ('The sashimi was the freshest and most tender I have ever tasted .', ['sashimi']), ('The only drawback is that this place is really expensive and 